## Imports

In [2]:
#!/usr/bin/env python3
import sevenbridges as sbg
import sys
import re
import os
from requests import request

config = sbg.Config(profile='cavatica')
api = sbg.Api(config=config)

### Probe cavatica for relevant files

In [15]:
def get_file_info(api, project, atype, out_fh):
    proj_tasks = api.tasks.query(project=project, status='COMPLETED').all()
    x = 0
    n = 5
    for task in proj_tasks:
        if x % 5 == 0:
            out_fh.flush()
        x += 1
        task_name = task.name
        task_id = task.id
        if atype == 'DNA':
            parts = task_name.split('-')
            parts2 = parts[3].split('_')
            tum_bs_id = 'BS_' + parts2[1]
            norm_bs_id = 'BS_' + parts2[3]
            outs = task.outputs['vep_annotated_maf'].name
            outs += ',' + task.outputs['cnv_result'].name
            out_fh.write('\t'.join((tum_bs_id, norm_bs_id, task_name, task_id, atype, outs)) + '\n')
        else:
            try:
                parts = task_name.split()
                parts2 = parts[0].split('-')
                bs_id = parts2[1]
            
                outs = task.outputs['result'][0].name
                out_fh.write('\t'.join((bs_id, 'NA', task_name, task_id, atype, outs)) + '\n')
            except:
                sys.stderr.write('Task ' + task_name +  ' with ID ' + task_id + ' failed to gather all info\n')
                sys.stderr.flush()

## Summarize Somatic project run on cavatica

In [16]:
dna_types = ['kfdrc-harmonization/sd-bhjxbdqk-07', 'kfdrc-harmonization/sd-bhjxbdqk-08']
rna_types = ['kfdrc-harmonization/sd-bhjxbdqk-06']
out_fh = open('/Users/brownm28/Documents/2018-Aug-15_cbttc_launch/portal_metadata/check_and_balances/audit/cavatica-up_audit/cbttc_cavatica_file_info.txt', 'w')
out_fh.write('T/CL BS ID\tNorm BS ID\tTask Name\tTask ID\tAnalyte Type\tRelevant Outputs\n')
for project in dna_types:
    get_file_info(api, project, 'DNA', out_fh)
for project in rna_types:
    get_file_info(api, project, 'RNA', out_fh)
out_fh.close()
sys.stderr.write('Complete\n')


Task Bam-to-fastqgz run - 08-14-18 20:42:29 with ID 110200d4-a10d-4d0c-95e0-8904ffa7a6d3 failed to gather all info
Task Bam-to-fastqgz run - 08-17-18 17:55:10 with ID de8bddb4-d632-4367-8fc2-d6b96007d966 failed to gather all info
Task RNAseq-split_bam_to_fastq with ID aeecb4b4-0ced-479d-955e-a618c819409a failed to gather all info
Complete
